<a href="https://colab.research.google.com/github/tomasz-soltysiak/NLP_IndianLanguage/blob/master/NLP_Nueral_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import matplotlib.patches as mpatches
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.model_selection import cross_val_score
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score,roc_curve,confusion_matrix,roc_auc_score, mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import re

In [8]:
df_train=pd.read_csv('/content/tamil_movie_reviews_train.csv',delimiter=',')
df_test=pd.read_csv('/content/tamil_movie_reviews_test.csv',delimiter=',')
df_train.head()

,ReviewId,ReviewInTamil,Rating
0,408,"தமிழ் சினிமாவில் ஒரு சிலர் மட்டுமே பணம், பிஸின...",4.00
1,107,கரு : சினிமா படப்பிடிப்புகளில் வெளிச்சம் பாய்ச...,2.00
2,319,தமிழ் சினிமாவில் தற்போது நாயகர்களுக்கு இணையாக ...,3.25
3,484,உலக அளவில் அனைத்து தரப்பினரையும் தன் நடிப்பால்...,2.25
4,204,"கரு : வில்லனின் கையாள் , வில்லன் செய்த நம்பிக்...",3.00


In [12]:
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
#stop_words = set(stopwords.words('')) 
porter=PorterStemmer()
lemmatizer = WordNetLemmatizer()
def clean_text(text):
    text = re.sub(r'[^\w\s]','',text, re.UNICODE)
    text = text.lower()
    text = [porter.stem(word) for word in text]
    #text = [lemmatizer.lemmatize(token) for token in text]
    #text = [lemmatizer.lemmatize(token, "v") for token in text]
    #text = [word for word in text if not word in stop_words]
    #text = " ".join(text)
    return text

df_train['ReviewInTamil1'] = df_train.ReviewInTamil.apply(lambda x: clean_text(x))
df_test['ReviewInTamil1'] = df_test.ReviewInTamil.apply(lambda x: clean_text(x))
df_train.head()







,ReviewId,ReviewInTamil,Rating,ReviewInTamil1
0,408,"தமிழ் சினிமாவில் ஒரு சிலர் மட்டுமே பணம், பிஸின...",4.00,"[த, ம, ழ, , ச, ன, ம, வ, ல, , ஒ, ர, , ச, ல, ..."
1,107,கரு : சினிமா படப்பிடிப்புகளில் வெளிச்சம் பாய்ச...,2.00,"[க, ர, , , ச, ன, ம, , ப, ட, ப, ப, ட, ப, ப, ..."
2,319,தமிழ் சினிமாவில் தற்போது நாயகர்களுக்கு இணையாக ...,3.25,"[த, ம, ழ, , ச, ன, ம, வ, ல, , த, ற, ப, த, , ..."
3,484,உலக அளவில் அனைத்து தரப்பினரையும் தன் நடிப்பால்...,2.25,"[உ, ல, க, , அ, ள, வ, ல, , அ, ன, த, த, , த, ..."
4,204,"கரு : வில்லனின் கையாள் , வில்லன் செய்த நம்பிக்...",3.00,"[க, ர, , , வ, ல, ல, ன, ன, , க, ய, ள, , , ..."


In [40]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers
max_features = 6000
tokenizer = Tokenizer(num_words=max_features)
scaler=StandardScaler()
tokenizer.fit_on_texts(df_train['ReviewInTamil1'])
list_tokenized_train = tokenizer.texts_to_sequences(df_train['ReviewInTamil1'])

tokenizer.fit_on_texts(df_test['ReviewInTamil1'])
list_tokenized_test = tokenizer.texts_to_sequences(df_test['ReviewInTamil1'])
maxlen = 130

X_train = pad_sequences(list_tokenized_train, maxlen=maxlen)
#X_train=np.array(scaler.fit_transform(X_train)).reshape(-1,1)
y_train = df_train['Rating']
#y_train=np.array(scaler.fit_transform(y_train))

X_test = pad_sequences(list_tokenized_test, maxlen=maxlen)
y_test = df_test['Rating']

embed_size = 128
model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])
batch_size = 100
epochs = 3
model.fit(X_train,y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Train on 384 samples, validate on 96 samples
Epoch 1/3
384/384 [==============================] - 5s 14ms/step - loss: 5.7596 - mean_squared_error: 5.7596 - val_loss: 5.5925 - val_mean_squared_error: 5.5925
Epoch 2/3
384/384 [==============================] - 1s 3ms/step - loss: 5.6153 - mean_squared_error: 5.6153 - val_loss: 5.4431 - val_mean_squared_error: 5.4431
Epoch 3/3
384/384 [==============================] - 1s 3ms/step - loss: 5.4573 - mean_squared_error: 5.4573 - val_loss: 5.2622 - val_mean_squared_error: 5.2622


In [46]:

len((np.array(scaler.fit_transform(X_train))))


480

In [17]:
y_pred=model.predict(X_test)
mean_squared_error(y_test,y_pred)

3.7218474126972754